In [22]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [48]:
df = pd.read_csv("matches.csv")

df.head()

/var/folders/x9/fxnv8bb96j99gbr28jn7scdc0000gn/T/ipykernel_80983/3816335167.py:1: DtypeWarning: Columns (136,141,143,145,146,148,149,151,152,154) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("matches.csv")


,id,home,away,date,year,time (utc),attendance,venue,league,part_of_competition,...,home_bench_10,home_bench_10_minute,home_bench_11_num,home_bench_11,home_bench_11_minute,away_bench_10_num,away_bench_10,away_bench_10_minute,away_starting_14_num,away_starting_14
0,18123,Charlton Athletic,Everton,"Saturday, August 18",2001,14:00,"20,451","The Valley, London, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18124,Derby County,Blackburn Rovers,"Saturday, August 18",2001,14:00,"28,236","Pride Park Stadium, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18125,Leeds United,Southampton,"Saturday, August 18",2001,14:00,"39,715","Elland Road, Leeds, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18126,Leicester City,Bolton Wanderers,"Saturday, August 18",2001,14:00,"19,987","Filbert Street, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18127,Liverpool,West Ham United,"Saturday, August 18",2001,14:00,"43,935","Anfield, Liverpool, England",2001-2002 Barclays Premier League,na,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
import re

df['result'] = np.where(df['home_score'] > df['away_score'], 1, np.where(df['home_score'] < df['away_score'], 2, 0))

def extraer_numero(texto):
    patron = r'\((\d+)\)'
    resultado = re.search(patron, texto)
    if resultado:
        return int(resultado.group(1))
    else:
        return None

columnas_deseadas = ['home','away','home_score', 'year', 'away_score', 'home_possession', 'away_possession', 'result', 'home_shotsSummary', 'away_shotsSummary', 'home_yellowCards', 'away_yellowCards', 'home_redCards', 'away_redCards']

df = df.filter(items=columnas_deseadas)

filtered_df = df[((df['home'] == 'Liverpool') & (df['away'] == 'Manchester United')) | ((df['home'] == 'Manchester United') & (df['away'] == 'Liverpool'))]

df_encoded = pd.get_dummies(filtered_df, columns=['home', 'away'])

print(df_encoded.head(10))

      home_score  year  away_score  result home_shotsSummary  \
109            3  2001           1       1             0 (0)   
228            0  2001           1       2             0 (0)   
535            1  2002           2       2            13 (7)   
691            4  2002           0       1            12 (9)   
876            1  2003           2       2            10 (4)   
1101           0  2003           1       2            11 (4)   
1198           2  2004           1       1            14 (9)   
1360           0  2004           1       2            13 (5)   
1571           0  2005           0       0             9 (3)   
1742           1  2005           0       1             7 (4)   

     away_shotsSummary  home_yellowCards  away_yellowCards  home_redCards  \
109              0 (0)               0.0               0.0            0.0   
228              0 (0)               1.0               2.0            0.0   
535              5 (4)               1.0               4.0      

In [38]:
import re
import pandas as pd

# Definir la función de extracción del número dentro de los paréntesis
def extraer_numero(texto):
    patron = r'\((\d+)\)'
    resultado = re.search(patron, texto)
    if resultado:
        return int(resultado.group(1))
    else:
        return None

# Filtrar las columnas deseadas
columnas_deseadas = ['home','away','home_score', 'year', 'away_score', 'home_possession', 'away_possession', 'result', 'home_shotsSummary', 'away_shotsSummary', 'home_yellowCards', 'away_yellowCards', 'home_redCards', 'away_redCards']
df = df.filter(items=columnas_deseadas)

# Aplicar la extracción del número dentro de los paréntesis a las columnas home_shotsSummary y away_shotsSummary
df['home_shotsSummary'] = df['home_shotsSummary'].apply(lambda x: extraer_numero(x) if pd.notnull(x) else None)
df['away_shotsSummary'] = df['away_shotsSummary'].apply(lambda x: extraer_numero(x) if pd.notnull(x) else None)

df.head(500)

,home,away,home_score,year,away_score,home_shotsSummary,away_shotsSummary,home_yellowCards,away_yellowCards,home_redCards,away_redCards
0,Charlton Athletic,Everton,1,2001,2,0.0,0.0,0.0,3.0,0.0,0.0
1,Derby County,Blackburn Rovers,2,2001,1,0.0,0.0,1.0,0.0,0.0,0.0
2,Leeds United,Southampton,2,2001,0,0.0,0.0,1.0,2.0,0.0,1.0
3,Leicester City,Bolton Wanderers,0,2001,5,0.0,0.0,4.0,3.0,0.0,0.0
4,Liverpool,West Ham United,2,2001,1,0.0,0.0,1.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
495,Blackburn Rovers,Aston Villa,0,2002,0,6.0,4.0,1.0,2.0,0.0,0.0
496,Leeds United,Everton,0,2002,1,5.0,7.0,3.0,3.0,0.0,0.0
497,Charlton Athletic,Sunderland,1,2002,1,12.0,7.0,1.0,4.0,0.0,0.0
498,Newcastle United,Middlesbrough,2,2002,0,13.0,9.0,2.0,2.0,0.0,1.0


In [50]:
k = 0.1
df_encoded['factor_of_importance'] = np.exp(k * (df_encoded['year'] - 2023))

df_encoded.head(50)

,home_score,year,away_score,result,home_shotsSummary,away_shotsSummary,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_Liverpool,home_Manchester United,away_Liverpool,away_Manchester United,factor_of_importance
109,3,2001,1,1,0 (0),0 (0),0.0,0.0,0.0,0.0,True,False,False,True,0.110803
228,0,2001,1,2,0 (0),0 (0),1.0,2.0,0.0,0.0,False,True,True,False,0.110803
535,1,2002,2,2,13 (7),5 (4),1.0,4.0,0.0,0.0,True,False,False,True,0.122456
691,4,2002,0,1,12 (9),5 (1),1.0,3.0,0.0,1.0,False,True,True,False,0.122456
876,1,2003,2,2,10 (4),6 (4),1.0,2.0,0.0,0.0,True,False,False,True,0.135335
1101,0,2003,1,2,11 (4),9 (4),2.0,1.0,0.0,0.0,False,True,True,False,0.135335
1198,2,2004,1,1,14 (9),6 (3),1.0,2.0,0.0,0.0,False,True,True,False,0.149569
1360,0,2004,1,2,13 (5),5 (2),2.0,4.0,0.0,1.0,True,False,False,True,0.149569
1571,0,2005,0,0,9 (3),4 (1),2.0,2.0,0.0,0.0,True,False,False,True,0.165299
1742,1,2005,0,1,7 (4),11 (7),2.0,3.0,0.0,0.0,False,True,True,False,0.165299


In [51]:
X = df_encoded.drop(['result', 'factor_of_importance'], axis=1)
X['factor_of_importance'] = df_encoded['factor_of_importance']
y = df_encoded['result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'criterion': ['gini', 'entropy']
}

rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

rf_classifier_best = RandomForestClassifier(random_state=42, **best_params)
rf_classifier_best.fit(X_train, y_train)

y_pred = rf_classifier_best.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)




# X = df_encoded.drop('result', axis=1)
# y = df_encoded['result']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 15],
#     'criterion': ['gini', 'entropy']
# }

# rf_classifier = RandomForestClassifier(random_state=42)

# grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

# rf_classifier_best = RandomForestClassifier(random_state=42, **best_params)
# rf_classifier_best.fit(X_train, y_train)

# y_pred = rf_classifier_best.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1 = f1_score(y_test, y_pred, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)



ValueError: 
All the 90 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 345, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py", line 1106, in check_X_y
    X = check_array(
      